In [1]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import platform

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 

In [2]:
from selenium import webdriver
import time

In [19]:
driver = webdriver.Chrome('driver/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

In [25]:
driver.find_element_by_id('electionType1').click()

In [28]:
driver.find_element_by_id('electionName').send_keys('제19대')

In [29]:
driver.find_element_by_id('electionCode').send_keys('대통령선거')

In [51]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [52]:
import re

In [53]:
def get_num(tmp):
    return float(re.split('\(', tmp)[0]. replace (',',''))

In [54]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [55]:
wait = WebDriverWait(driver, 10)

In [60]:
def move_sido(name):
    element = driver.find_elements_by_id('cityCode')
    element.send_keys(name)
    make_xpath = '''//*[@id = searchBtn]'''
    wait.until(EC.element_wait_to_be_clickable((By.xpath.make_xpath)))
    driver.find_element_by_xpath(make_xpath).click()